In [2]:
import pandas as pd
import numpy as np

# 1. 파일 불러오기
df = pd.read_excel("merged_patient_data_with_NLR.xlsx")

# 2. CACS가 존재하는 환자만 필터링
df = df.dropna(subset=["CACS"]).copy()

# 3. 'age' 문자열 → 숫자 변환 ("73yrs" → 73)
df["age"] = df["age"].astype(str).str.replace("yrs", "", regex=False)
df["age"] = pd.to_numeric(df["age"], errors="coerce")

# 4. 문자열 포함된 수치형 (TG, CRP, eGFR) → 숫자로 변환
def clean_numeric(x):
    try:
        return float(str(x).replace(">", "").replace("<", "").strip())
    except:
        return np.nan

for col in ["CRP", "eGFR", "TG"]:
    df[col] = df[col].apply(clean_numeric)

# 5. 분석에 사용할 변수 선택
features = ["BMI", "age", "SBP", "DBP", "CRP", "eGFR", 
            "HDL", "LDL", "Lymp", "Neu", "TG", "TC", "NLR"]

df_model = df[features + ["CACS"]].copy()

# 6. 중앙값(median)으로 결측치 대체 (imputation)
df_model = df_model.fillna(df_model.median(numeric_only=True))

# 7. 결과 확인
print(f"🧮 최종 분석 가능한 환자 수 (imputation 적용): {len(df_model)}명")
display(df_model.head())

# 8. 저장 (선택적)
df_model.to_excel("CACS_model_ready_imputed.xlsx", index=False)
print("✅ 전처리 완료된 파일 저장: CACS_model_ready_imputed.xlsx")

# Neu, Lymp 변수 제거
df_model = df_model.drop(columns=["Neu", "Lymp"])

# 1. 불러오기
df = pd.read_excel("CACS_model_ready_imputed.xlsx")

# 2. Neu, Lymp 제거
df = df.drop(columns=["Neu", "Lymp"])

# 3. 저장
df.to_excel("CACS_model_ready_NLR_only.xlsx", index=False)

print("✅ 'Neu'와 'Lymp' 제거 완료. 저장된 파일: CACS_model_ready_NLR_only.xlsx")


🧮 최종 분석 가능한 환자 수 (imputation 적용): 2801명


,BMI,age,SBP,DBP,CRP,eGFR,HDL,LDL,Lymp,Neu,TG,TC,NLR,CACS
1,22.7732,70.0,115.0,65.0,0.990,77.8,31.0,69.0,34.0,51.6,137.0,118.0,1.517647,1.0
4,25.1486,65.0,133.0,78.0,0.100,60.0,51.0,81.0,31.5,57.8,112.0,151.0,1.839858,21.0
5,23.5883,76.0,135.0,81.0,0.038,65.6,68.0,69.0,25.0,61.1,78.0,144.0,2.444000,57.0
6,27.0552,61.0,149.0,95.0,2.950,69.0,51.0,81.0,37.7,49.8,112.0,122.0,1.320955,87.0
7,25.1486,65.0,133.0,78.0,0.100,60.0,51.0,81.0,31.5,57.8,112.0,151.0,1.839858,234.0


✅ 전처리 완료된 파일 저장: CACS_model_ready_imputed.xlsx
✅ 'Neu'와 'Lymp' 제거 완료. 저장된 파일: CACS_model_ready_NLR_only.xlsx
